In [ ]:
import os, sys
src_paths = [os.path.abspath("../"), os.path.abspath("../point-to-pixel-mapping")]
for src_path in src_paths:
    if src_path not in sys.path:
        sys.path.append(src_path)

from datasets.kitti_dataset import KittiDataset

In [ ]:
kitti_dataset_path = '/Users/laurenzheidrich/Downloads/fused_dataset'
sequence = '00'
sam_labels_path = '/Users/laurenzheidrich/Downloads/fused_dataset/vfm-labels/sam/00'

dataset = KittiDataset(kitti_dataset_path, 
                       sequence, 
                       sam_labels_path)

labels_folder = '/Users/laurenzheidrich/Downloads/fused_dataset/sequences/00/labels'

In [ ]:
from segmentation.reprojection_segmentation import ReprojectionSegmentation
from segmentation.dbscan_segmentation import DbscanSegmentation
from segmentation.spectral_clustering_segmentation import SpectralClusteringSegmentation
from segmentation.ncut_segmentation import nCutSegmentation

visual_segmentation = ReprojectionSegmentation(dataset)
geometrical_segmentation = DbscanSegmentation(dataset)
#spectral_segmentation = SpectralClusteringSegmentation(dataset, n_clusters=40, lower_bound=0, upper_bound=25)
ncut_segmentation = nCutSegmentation(dataset, ncut_threshold=0.001, lower_bound=6, upper_bound=45)

### Run baselines

In [ ]:
import os
import numpy as np

from tqdm import tqdm

indices = []
visual_labels = []
geometrical_labels = []
spectral_labels = []
ncut_labels = []
gt_labels = []


for i in tqdm(range(0, len(dataset.poses), 1000)):
    indices.append(i)
    visual_labels.append(visual_segmentation.segment_instances(i))
    geometrical_labels.append(geometrical_segmentation.segment_instances(i))
    #spectral_labels.append(spectral_segmentation.segment_instances(i))
    ncut_labels.append(ncut_segmentation.segment_instances(i))
    labels_file_path = os.path.join(labels_folder, '{}.label'.format(str(i).zfill(6)))
    gt_labels.append(np.fromfile(labels_file_path, dtype=np.uint32))


### Visualize baselines

In [ ]:
import random

def generate_random_colors(N):
    colors = []
    for _ in range(N):
        colors.append([random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)])

    return colors

random_colors = generate_random_colors(500)

In [ ]:
import numpy as np
import open3d as o3d

def color_points_by_labels(points, labels):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    colors = []

    for i in range(labels.shape[0]):
        try:
            colors.append(random_colors[int(labels[i]) + 1])
        except:
            colors.append(random_colors[1])
            print(int(labels[i]) + 1)

    pcd.colors = o3d.utility.Vector3dVector(np.vstack(colors) / 255)

    return pcd

In [ ]:
index = 0
points = dataset.get_point_cloud(indices[index])

In [ ]:
#o3d.visualization.draw_geometries([color_points_by_labels(points, visual_labels[index])])
o3d.io.write_point_cloud("pcd_visual_labels.pcd", color_points_by_labels(points, visual_labels[index]), write_ascii=False, compressed=False, print_progress=False)

In [ ]:
#o3d.visualization.draw_geometries([color_points_by_labels(points, gt_labels[index])])
o3d.io.write_point_cloud("pcd_gt_labels.pcd", color_points_by_labels(points, gt_labels[index]), write_ascii=False, compressed=False, print_progress=False)

In [ ]:
#o3d.visualization.draw_geometries([color_points_by_labels(points, spectral_labels[index])])
o3d.io.write_point_cloud("pcd_spectral_labels.pcd", color_points_by_labels(points, spectral_labels[index]), write_ascii=False, compressed=False, print_progress=False)

In [ ]:
#o3d.visualization.draw_geometries([color_points_by_labels(points, geometrical_labels[index])])
o3d.io.write_point_cloud("pcd_geometrical_labels.pcd", color_points_by_labels(points, geometrical_labels[index]), write_ascii=False, compressed=False, print_progress=False)

In [ ]:
#o3d.visualization.draw_geometries([color_points_by_labels(points, ncut_labels[index])])
o3d.io.write_point_cloud("pcd_ncut_labels.pcd", color_points_by_labels(points, ncut_labels[index]), write_ascii=False, compressed=False, print_progress=False)

### Evaluation

In [ ]:
import instanseg
from instanseg.metrics import precision, recall, fScore

instanseg.metrics.constants.IOU_THRESHOLD_FULL = 0.5
tp_condition = 'iou'

In [ ]:
import pandas as pd

results_dict = []
for i in tqdm(range(len(spectral_labels))):
    res = {
        'index': i,
        'method': 'visual',
        'precision': precision(visual_labels[i], gt_labels[i], tp_condition),
        'recall': recall(visual_labels[i], gt_labels[i], tp_condition),
        'fscore': fScore(visual_labels[i], gt_labels[i], tp_condition)
        
    }
    results_dict.append(res)

    res = {
        'index': i,
        'method': 'geometrical',
        'precision': precision(geometrical_labels[i], gt_labels[i], tp_condition),
        'recall': recall(geometrical_labels[i], gt_labels[i], tp_condition),
        'fscore': fScore(geometrical_labels[i], gt_labels[i], tp_condition)
    }
    results_dict.append(res)

    '''
    res = {
        'index': i,
        'method': 'spectral',
        'precision': precision(spectral_labels[i], gt_labels[i], tp_condition),
        'recall': recall(spectral_labels[i], gt_labels[i], tp_condition),
        'fscore': fScore(spectral_labels[i], gt_labels[i], tp_condition)
    }
    results_dict.append(res)
    '''

    res = {
        'index': i,
        'method': 'ncut',
        'precision': precision(ncut_labels[i], gt_labels[i], tp_condition),
        'recall': recall(ncut_labels[i], gt_labels[i], tp_condition),
        'fscore': fScore(ncut_labels[i], gt_labels[i], tp_condition)
    }
    results_dict.append(res)


In [ ]:
df = pd.DataFrame.from_records(results_dict, index=np.arange(len(results_dict)))

In [ ]:
print(df.head())

In [ ]:
import seaborn as sns

In [ ]:
sns.lineplot(df, x='index', y='recall', hue='method')

In [ ]:
sns.lineplot(df, x='index', y='precision', hue='method')

In [ ]:
sns.lineplot(df, x='index', y='fscore', hue='method')